In [1]:
using RD

In [ ]:
function build_model_state(L)
    g,posx,posy = RD.gen_hex_lattice(L)
    N = length(posx)
    memb0 = zeros(Int,N,4)	
    unit_length = 1.0

    theta = 0.2126944621086619#Stot/V


    Ac = unit_length^2#area associated to each cell, set unit lengthscale
    Stot = N*Ac #area of each cell is assumed to be 1, setting unit lengthscale
    V = Stot/theta
    d_timescale = 1.0#this sets unit timescale
    dA = d_timescale
    dB = d_timescale
    dEA = d_timescale
    dEB = d_timescale
    #rates in the theory (do not correspond excatly to those used in the simulations), some slight dimensional changes are needed
    kAa_th = 1.0
    kAd_th = 1.0*kAa_th
    kAc_th = 1.0
    kBa_th = 1.0
    kBd_th = 1.0*kBa_th
    kBc_th = logrange(logrange(0.1,10,17)[4],logrange(0.1,10.0,17)[14],23)[13]*kAc_th
    KMM_th = 1.0
    #rates to implement
    kAc = kAc_th
    kBc = kBc_th
    kAa = kAa_th/V
    kAd = kAd_th
    kBa = kBa_th/V
    kBd = kBd_th
    KMM = KMM_th*Ac

    M = Model(g, posx, posy, dA, dB, dEA, dEB, kAc, kBc, kAa, kAd, kBa, kBd, KMM, 0.0)

    N = length(M)
    totmol = N * 10
    totA = 0.5 * totmol
    totB = 0.5 * totmol
    theta = 0.2126944621086619#Stot/V

    nA, nB, nEA, nEB = fill(0,N), fill(0,N), fill(0,N), fill(0,N)

    nA[rand(1:N, floor(Int, totA))] .+= 1
    nB[rand(1:N, floor(Int, totB))] .+= 1	
    EA_tot_n = floor(Int, 0.1*N)
    EB_tot_n = floor(Int, 0.1*N)
    EA_mem = floor(Int, EA_tot_n*(theta/(kAd_th/kAa_th))*(totA/Stot)/(1+((theta/(kAd_th/kAa_th))*(totA/Stot))))
    EB_mem = floor(Int, EB_tot_n*(theta/(kBd_th/kBa_th))*(totB/Stot)/(1+((theta/(kBd_th/kBa_th))*(totB/Stot))))
    nEA[rand(1:N, EA_mem)] .+= 1
    nEB[rand(1:N, EB_mem)] .+= 1
    
    cytoEA = EA_tot_n - EA_mem
    cytoEB = EB_tot_n - EB_mem

    s = State(nA, nB, nEA, nEB, Ref(cytoEA), Ref(cytoEB))
    M,s
end

build_model_state (generic function with 1 method)

In [13]:
using GraphPlot

T = 20000.0
L = 10

M,s = build_model_state(L)
gplot(M.g, M.posx, M.posy)


MethodError: MethodError: no method matching rand(::UnitRange{Int64}, ::Float64)
The function `rand` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  rand(!Matched::Random.AbstractRNG, ::Any, !Matched::Integer, !Matched::Integer...)
   @ Random ~/src/julia/build1.11/usr/share/julia/stdlib/v1.11/Random/src/Random.jl:284
  rand(::Any, !Matched::Integer, Integer...)
   @ Random ~/src/julia/build1.11/usr/share/julia/stdlib/v1.11/Random/src/Random.jl:285
  rand(!Matched::Random.AbstractRNG, ::Any)
   @ Random ~/src/julia/build1.11/usr/share/julia/stdlib/v1.11/Random/src/Random.jl:255
  ...


In [14]:
rand(1:10, 2)

2-element Vector{Int64}:
 10
  9

In [18]:
p = ProgressMeasurer(T)
m = Measurer(M; times=T/2:1000:T)
stats = (s,t)->(p(s,t); m(s,t))

run_RD!(s, M, T; stats = m)

starting simulation, 3200 events in the queue
T = 10000.000024582867 and <ϕ>/c = 0.115
Binder cumulant: 0.05563478066875904
T = 11000.00022911609 and <ϕ>/c = 0.159
Binder cumulant: -0.06648522960693581
T = 12000.000221690567 and <ϕ>/c = 0.157
Binder cumulant: -0.10265509966739361
T = 13000.000155410558 and <ϕ>/c = 0.2405
Binder cumulant: 0.05613624339006007
T = 14000.000909320896 and <ϕ>/c = 0.1275
Binder cumulant: -0.0033852359087931383
T = 15000.000300078324 and <ϕ>/c = 0.219
Binder cumulant: 0.06686351945020153
T = 16000.00000025183 and <ϕ>/c = 0.2015
Binder cumulant: -0.07134795185866016
T = 17000.000029274972 and <ϕ>/c = 0.1855
Binder cumulant: 0.03080502363072879
T = 18000.000160070595 and <ϕ>/c = 0.153
Binder cumulant: 0.09711570901809152
T = 19000.000339573042 and <ϕ>/c = 0.1455
Binder cumulant: 0.0030491335468404435


In [19]:
include("../src/RDold.jl")
include("../src/run_tests.jl")

0.2126944621086619


estimated rho0 =1.1397725198066269
estimated rho+ =3.1269446210866194
estimated rho- =0.3198009946375379
estimated k =0.1
bistability region between 0.09090909090909091 and 2.0
physical region between 0.36450038549469216 and 3.5640055500716743
estimatted phi_eq/c= 0.12674377776054654
now run reaction diffusion
T=0.0 and <phi>/c=0.0
Binder cumulant: -0.052247473099695796
T=10.0 and <phi>/c=0.00696
Binder cumulant: -0.03992643697076925
T=20.0 and <phi>/c=0.01792
Binder cumulant: -0.03108421500120806
T=30.0 and <phi>/c=0.02216
Binder cumulant: -0.014558449344613145
T=40.0 and <phi>/c=0.03544
Binder cumulant: -0.017768544999159808
T=50.0 and <phi>/c=0.05096
Binder cumulant: 0.002373934076744688
T=60.0 and <phi>/c=0.0612
Binder cumulant: -0.004122973348759107
T=70.0 and <phi>/c=0.07688
Binder cumulant: 0.018986828581408655
T=80.0 and <phi>/c=0.08568
Binder cumulant: -0.020695207843581365
T=90.0 and <phi>/c=0.10272
Binder cumulant: 0.058343365234734934
T=100.0 and <phi>/c=0.11216
Binder cumu

LoadError: LoadError: InterruptException:
in expression starting at /home/ab/src/RD.jl/src/run_tests.jl:105